In [1]:
# Script to Prep the data from SMES and Weather for usability and to github

In [1]:
import pandas as pd
import numpy as np
import datetime

# Weather Data

# Flight Data

In [123]:
df_flight_raw = pd.read_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\data.csv')

In [124]:
df_flight_raw.head()

,airport,track,stid,call_sign,time,event,status,latitude,longitude,altitude,departure_airport,destination_airport
0,KMIA,2088,9375251,UAL2081,2020-01-01 00:00:07,on,onsurface,25.80386,-80.27901,12.50,KORD,KMIA
1,KPIT,3534,339554,SWA4052,2020-01-01 00:00:08,off,airborne,40.50235,-80.26791,1493.75,NaN,NaN
2,KMSY,4057,68290,SWA5830,2020-01-01 00:00:08,on,onsurface,29.99697,-90.28297,43.75,NaN,NaN
3,KMEM,3263,515627,AAL1996,2020-01-01 00:00:09,on,onsurface,35.05050,-89.98753,381.25,KDFW,KMEM
4,KDFW,683,1390176,AAL2346,2020-01-01 00:00:11,on,onsurface,32.87883,-97.02618,668.75,KORD,KDFW


In [125]:
df_airport = df_flight_raw[['airport','latitude','longitude','altitude']]

In [132]:
df_flight = df_flight_raw[['airport','stid','call_sign','time','event','status','departure_airport','destination_airport']].copy()

In [133]:
# Making Duplicate Data
df_flight_dup = df_flight.sample(frac = .08) 

# Making super early flights
df_flight_early = df_flight.sample(frac = .4) 
df_flight_early['time'] = pd.to_datetime(df_flight_early['time']) - datetime.timedelta(days=(365))

# Making Wrong Case Data
df_flight['test_temp'] = df_flight['airport'].str.lower()
df_flight['airport'] = np.where(df_flight['status'] == 'onramp', df_flight['test_temp'], df_flight['airport'])
df_flight = df_flight.drop(columns = ['test_temp'])

# Fake Null Values
df_flight.loc[df_flight.sample(frac = .02).index, 'event'] = None 

list_df = [df_flight, df_flight_dup, df_flight_early]

df_flight_final = pd.concat(list_df)

In [134]:
df_flight_final = df_flight_final[['airport','call_sign','time','event','departure_airport','destination_airport']]

In [135]:
df_flight_final = df_flight_final.sample(frac = .75) 

In [136]:
df_flight_final.shape

(2763193, 6)

In [137]:
df_flight_final.to_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\flight_data_test.csv', index = False)
df_airport.to_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\airport_data.csv', index = False)

In [138]:
def compress(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = compress(df_flight_final)
train_df.info(memory_usage='deep')

Mem. usage decreased to 147.57 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2763193 entries, 1588919 to 2092521
Data columns (total 6 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   airport              object
 1   call_sign            object
 2   time                 object
 3   event                object
 4   departure_airport    object
 5   destination_airport  object
dtypes: object(6)
memory usage: 1.0 GB


## Making the rest of datasets

In [139]:
# Make Complete Dataset

import pandas as pd
df = pd.read_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\flight_data_test.csv')
index = df['departure_airport'].notnull()
new_df = df[index].copy()
on_index = (new_df['event'].isnull()) & (new_df['airport'] == new_df['departure_airport'])
new_df.loc[on_index,'event'] = 'Off'
off_index = (new_df['event'].isnull()) & (new_df['airport'] == new_df['destination_airport'])
new_df.loc[off_index,'event'] = 'On'
index = new_df['event'].notnull()
complete_df = new_df[index]
complete_df.to_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\complete_data_test.csv', index=False)

In [140]:
# Make Clean Dataset

import pandas as pd
import numpy as np
import datetime

filepath = r'C:\Users\jpau\Documents\Projects\Airforce\data\complete_data_test.csv'
df = pd.read_csv(filepath)
dedup_df = df.drop_duplicates()
index = dedup_df['time'] >= '2020-01-01'
timed_df = dedup_df[index].copy()
timed_df['airport'] = timed_df['airport'].str.upper()
index = ((timed_df['destination_airport'] == timed_df['airport']) & (timed_df['event'] == 'on')) | ((timed_df['departure_airport'] == timed_df['airport']) & (timed_df['event'] == 'off'))
clean_df = timed_df[index].copy()
clean_df.to_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\clean_data_test.csv', index=False)

In [141]:
# Make eda Dataset

import pandas as pd
filepath = r'C:\Users\jpau\Documents\Projects\Airforce\data\clean_data_test.csv'
df = pd.read_csv(filepath)
df['type_of_flight'] = 'local'
df.loc[(df['destination_airport'].str[0] != 'K') | (df['departure_airport'].str[0] != 'K'),'type_of_flight'] = 'international'
df['airline'] = df['call_sign'].str[:3]
df.to_csv(r'C:\Users\jpau\Documents\Projects\Airforce\data\eda_data_test.csv', index = False)

In [142]:
def compress(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = compress(df)
train_df.info(memory_usage='deep')

Mem. usage decreased to 103.58 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696973 entries, 0 to 1696972
Data columns (total 8 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   airport              object
 1   call_sign            object
 2   time                 object
 3   event                object
 4   departure_airport    object
 5   destination_airport  object
 6   type_of_flight       object
 7   airline              object
dtypes: object(8)
memory usage: 820.1 MB
